In [1]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

/opt/homebrew/Caskroom/miniconda/base/envs/tf-fcc-ml-4/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.13.0


In [9]:
# get data files
# !wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
# !wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

# Set random seed for reproducibility
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Load the dataset
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [10]:
train_df = pd.read_csv(train_file_path, sep='\t', names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', names=["label", "message"])

# Preprocess labels
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Prepare datasets
train_labels = train_df.pop('label')
test_labels = test_df.pop('label')

# Define the TextVectorization layer
max_features = 1000
max_length = 1000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=max_length)

# Fit the TextVectorization layer to the training text
vectorize_layer.adapt(train_df['message'].values)

# Vectorize the messages
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

train_ds = tf.data.Dataset.from_tensor_slices((train_df['message'].values, train_labels.values)).batch(32).map(vectorize_text)
test_ds = tf.data.Dataset.from_tensor_slices((test_df['message'].values, test_labels.values)).batch(32).map(vectorize_text)

In [11]:
# Define the model with Bidirectional LSTM
model = Sequential([
    Embedding(max_features + 1, 64, mask_zero=True),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          64064     
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                

In [12]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Setup EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, mode='max', restore_best_weights=True)

# Train the model
model.fit(train_ds, epochs=10, validation_data=test_ds, callbacks=[early_stop], verbose=1)

Epoch 1/10
131/131 [==============================] - 97s 685ms/step - loss: 0.2348 - accuracy: 0.9299 - val_loss: 0.0725 - val_accuracy: 0.9792
Epoch 2/10
131/131 [==============================] - 89s 682ms/step - loss: 0.0508 - accuracy: 0.9864 - val_loss: 0.0546 - val_accuracy: 0.9828
Epoch 3/10
131/131 [==============================] - 91s 698ms/step - loss: 0.0290 - accuracy: 0.9919 - val_loss: 0.0545 - val_accuracy: 0.9842
Epoch 4/10
131/131 [==============================] - 90s 688ms/step - loss: 0.0151 - accuracy: 0.9959 - val_loss: 0.0526 - val_accuracy: 0.9856
Epoch 5/10
131/131 [==============================] - 89s 678ms/step - loss: 0.0119 - accuracy: 0.9969 - val_loss: 0.0559 - val_accuracy: 0.9856
Epoch 6/10
131/131 [==============================] - 91s 694ms/step - loss: 0.0121 - accuracy: 0.9971 - val_loss: 0.0856 - val_accuracy: 0.9806
Epoch 7/10
131/131 [==============================] - 93s 709ms/step - loss: 0.0217 - accuracy: 0.9933 - val_loss: 0.0638 - val_ac

In [13]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_text = vectorize_layer(tf.expand_dims(pred_text, -1))
    prediction = model.predict(pred_text)[0][0]
    label = "spam" if prediction >= 0.5 else "ham"
    # print(f"Message: {pred_text}\nPredicted: {label} ({prediction:.4f})")
    return [prediction, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 3s 3s/step
[1.22565625e-05, 'ham']


In [14]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 52ms/step
You passed the challenge. Great job!
